# Synthetic Traces Plotting/Analysis

The real traces are taken from a dataset found on Kaggle (and GitHub). During the experiment, there are also synthetic traces that are generated on the fly based on some distribution or function. This notebook explores the latter.

Note that there are some limitations:

* Each episode has a length of 288 steps (each step represents 5 minutes in a 24 hour period).
* In each step, there can be a maximum of 150 incoming requests to a node.

In [ ]:
# Common imports.
from pathlib import Path

%matplotlib widget
import base

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import ipywidgets

## Sinusoidal traces

## Generation method

In [ ]:
rng = np.random.default_rng(seed=42)  # RNG used to generate the requests.
min_reqs, max_reqs = 0, 150  # Range of the generated requests.
max_steps = 288  # Steps of a single episode (see the DFaaS env).

In [ ]:
# Generates the requests for a single agent.
def synthetic_sinusoidal(
    average_requests=74,
    amplitude_requests=72,
    noise_ratio=0.1,
    function=None,
    seed=0,
):
    steps = np.arange(max_steps)

    if function is None:
        # Must generate a different RNG to do not alter the RNG
        # used to sample.
        fn_rng = np.random.default_rng(seed=seed)
        function = fn_rng.choice([np.sin, np.cos])

    rng = np.random.default_rng(seed=seed)
    average_requests = np.clip(rng.normal(loc=70), 60, 80)

    # Sample a period with a fixed interval.
    period = rng.uniform(5, high=30)

    # Sample the function.
    base_input = average_requests + amplitude_requests * function(steps / period)

    # Add some noise.
    noisy_input = base_input + noise_ratio * rng.normal(
        0, amplitude_requests, size=max_steps
    )
    requests = np.asarray(noisy_input, dtype=np.int32)

    # Clip the excess values respecting the minimum and maximum values
    # for the input requests observation.
    np.clip(requests, min_reqs, max_reqs, out=requests)

    return requests

### Plotting of a generic trace

In [ ]:
def plot_sinusoidal_trace():
    avg_reqs = ipywidgets.IntSlider(
        value=74,
        min=0,
        max=200,
        description="Average requests [0, 200]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    ampl_reqs = ipywidgets.IntSlider(
        value=60,
        min=0,
        max=200,
        description="Amplitude requests [0, 200]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    noise = ipywidgets.FloatSlider(
        value=0.1,
        min=0,
        max=2,
        description="Noise [0, 2]:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    function = ipywidgets.Dropdown(
        options=[("None", None), ("sin", np.sin), ("cos", np.cos)],
        index=0,
        description="Function:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )
    seed = ipywidgets.BoundedIntText(
        value=42,
        min=0,
        max=np.iinfo(np.uint32).max,
        description="Seed:",
        style={"description_width": "initial"},
        layout=ipywidgets.Layout(width="500px"),
    )

    with plt.ioff():
        fig_invocs = plt.figure(layout="constrained")
        fig_dist = plt.figure(layout="constrained")
    fig_invocs.canvas.header_visible = False
    ax_invocs = fig_invocs.subplots()
    fig_dist.canvas.header_visible = False
    ax_dist = fig_dist.subplots()

    def make_plot():
        # Generate a new trace.
        trace = synthetic_sinusoidal(
            average_requests=avg_reqs.value,
            amplitude_requests=ampl_reqs.value,
            noise_ratio=noise.value,
            function=function.value,
            seed=seed.value,
        )

        # Make the invocations plot.
        ax_invocs.clear()
        ax_invocs.plot(trace)

        ax_invocs.set_title(f"Function invocations")
        ax_invocs.set_ylabel("Invocations")
        ax_invocs.yaxis.set_major_locator(ticker.MultipleLocator(10))
        ax_invocs.set_ylim(bottom=min_reqs, top=max_reqs + 5)
        ax_invocs.set_xlabel("Step")
        ax_invocs.xaxis.set_major_locator(ticker.MultipleLocator(25))

        ax_invocs.grid(axis="both")
        ax_invocs.set_axisbelow(True)

        # Make the invocations distribution plot.
        ax_dist.clear()

        # The X-axis (freqs_idx) and the values (freqs) must have the same length.
        freqs = pd.Series(trace).value_counts()
        freqs_idx = np.arange(freqs.index.max() + 1)
        freqs = freqs.reindex(range(freqs.index.max() + 1), fill_value=0)
        ax_dist.bar(freqs_idx, freqs)

        ax_dist.set_title(f"Function invocations distribution")
        ax_dist.set_ylabel("Frequency")
        ax_dist.set_xlabel("Invocation")
        # Force the X-axis to have integer values (float by default).
        ax_dist.xaxis.get_major_locator().set_params(integer=True)

        ax_dist.grid(axis="both")
        ax_dist.set_axisbelow(True)

        fig_dist.canvas.draw_idle()
        fig_invocs.canvas.draw_idle()
        fig_invocs.canvas.flush_events()
        fig_dist.canvas.flush_events()

    # Make the initial plot with the default values.
    make_plot()

    # Link the input widgets to the plotting function.
    avg_reqs.observe(lambda change: make_plot(), names="value")
    ampl_reqs.observe(lambda change: make_plot(), names="value")
    noise.observe(lambda change: make_plot(), names="value")
    function.observe(lambda change: make_plot(), names="value")
    seed.observe(lambda change: make_plot(), names="value")

    # Callback when the user press the reset button.
    def reset(widget, default_value):
        widget.index = 0
        make_plot()

    # Create the reset buttons and link them to the correct widget.
    avg_reqs_reset = ipywidgets.Button(description="Reset", icon="undo")
    avg_reqs_reset.on_click(lambda _: reset(avg_reqs, 76))
    ampl_reqs_reset = ipywidgets.Button(description="Reset", icon="undo")
    ampl_reqs_reset.on_click(lambda _: reset(ampl_reqs, 60))
    noise_reset = ipywidgets.Button(description="Reset", icon="undo")
    noise_reset.on_click(lambda _: reset(noise, 0.1))
    function_reset = ipywidgets.Button(description="Reset", icon="undo")
    function_reset.on_click(lambda _: reset(function, 0))

    avg_reqs_reset.click()

    # Create the last button to generate a random seed.
    def random_seed():
        iinfo = np.iinfo(np.uint32)
        seed.value = rng.integers(0, high=iinfo.max, size=1)[0].item()

    random_seed_btn = ipywidgets.Button(description="Random", icon="shuffle")
    random_seed_btn.on_click(lambda _: random_seed())

    inputs = ipywidgets.VBox(
        [
            ipywidgets.HBox([avg_reqs, avg_reqs_reset]),
            ipywidgets.HBox([ampl_reqs, ampl_reqs_reset]),
            ipywidgets.HBox([noise, noise_reset]),
            ipywidgets.HBox([function, function_reset]),
            ipywidgets.HBox([seed, random_seed_btn]),
        ]
    )

    return ipywidgets.AppLayout(
        header=inputs,
        center=ipywidgets.VBox([fig_invocs.canvas, fig_dist.canvas]),
        pane_heights=[0.6, 5, 0],
    )


plot_sinusoidal_trace()